In [1]:
!python -V
!pip show tensorflow

Python 3.6.5
Name: tensorflow
Version: 1.8.0
Summary: TensorFlow helps the tensors flow
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: c:\users\b\desktop\direct_dcrnn\env\lib\site-packages
Requires: six, tensorboard, grpcio, gast, protobuf, absl-py, termcolor, astor, numpy, wheel
Required-by: 


In [2]:
import io
import os
import sys

import argparse
import numpy as np

import glob
import yaml

import tensorflow as tf
from lib.utils import load_graph_data
from model.dcrnn_supervisor import DCRNNSupervisor

c:\users\b\desktop\adversarial_traffic_flow\adversarial_gc_grnn\env\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\b\desktop\adversarial_traffic_flow\adversarial_gc_grnn\env\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\b\desktop\adversarial_traffic_flow\adversarial_gc_grnn\env\lib\site-packages\tensorflow\python\framework\dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("q

In [3]:
best_model_dir = './best_model/'
name_splits = glob.glob(best_model_dir+'*.index')[0].split('.')
model_filename = '.'+name_splits[1] + '.'+ name_splits[2]
print("The model being used for predictions is:",model_filename)

The model being used for predictions is: ./best_model\models-462.8403-63427


In [4]:
#print("Both below are required only in training:")
predictions_file ="GCGRNN_volume_150.csv"
#print("The predictions at the end of this notebook will be available in :", predictions_file)

ground_truths_file = "y_truth_GCGRNN_volume_150.csv"
#print("The ground truth of those predictions will be available in :", ground_truths_file)

In [5]:
# Configuration YAML for the DCRNN model

configuration = { 'base_dir': 'results',
                  'log_level':'INFO',
                  'data':{ 'batch_size': 32,
                           'dataset_dir': './data/',
                           'graph_pkl_filename': './helper_files/adj_mat_volume.pkl',
                           'test_batch_size': 64,
                           'val_batch_size': 64},
                    'model':{'cl_decay_steps': 2000,
                             'filter_type': 'dual_random_walk',
                             'horizon': 12,
                             'input_dim': 1,
                             'l1_decay': 0 ,
                             'max_diffusion_step': 1,
                             'num_nodes': 150,
                             'num_rnn_layers': 2,
                             'output_dim': 1,
                             'rnn_units': 64,
                             'seq_len': 12,
                             'use_curriculum_learning': True},
                    'train':{ 'base_lr': 0.01,
                              'dropout': 0,
                              'epoch': 64,
                              'epochs': 300,
                              'epsilon': 0.001,
                              'global_step': 18655,
                              'groundtruth_file': ground_truths_file,
                             'log_dir': 'results',
                              'lr_decay_ratio': 0.1,
                              'max_grad_norm': 5,
                              'max_to_keep': 100,
                              'min_learning_rate': 2.0e-06,
                              'model_filename': model_filename,
                              'optimizer': 'adam',
                              'patience': 50,
                              'preds_file': predictions_file,
                              'steps':'[20]',
                              'test_every_n_epochs': 1}
                }
 
# Write YAML file
config_file_name = './best_model/config_for_pre_trained_dcrnn.yaml'
with io.open(config_file_name, 'w', encoding='utf8') as outfile:
    yaml.dump(configuration, outfile, default_flow_style=False, allow_unicode=True)

In [6]:
def run_dcrnn(config_file, output_file, use_gpu = True):
    
    # safely load the config yaml file
    with open(config_file, 'r') as f:
        try: 
            cfile = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
            
    tf_config = tf.ConfigProto()
    if use_gpu:
        tf_config = tf.ConfigProto(device_count={'GPU': 0})
    tf_config.gpu_options.allow_growth = True
    
    graph_pkl_filename = cfile['data']['graph_pkl_filename']
    _, _, adj_mx = load_graph_data(graph_pkl_filename)
    
    with tf.Session(config=tf_config) as sess:
        
        supervisor = DCRNNSupervisor(adj_mx=adj_mx, **cfile)
        
        supervisor.load(sess, cfile['train']['model_filename'])
        
        outputs = supervisor.evaluate(sess)
        
        np.savez_compressed(output_file, **outputs)
        print('Predictions saved as {}.'.format(output_file))

In [7]:
output_filename = './best_model/best_model_output.npz'

# If you do not have a GPU, set the use_gpu option to False
run_dcrnn(config_file_name,output_filename, use_gpu = True)  

2021-03-19 15:29:58,345 - INFO - Log directory: results
2021-03-19 15:29:58,346 - INFO - {'base_dir': 'results', 'data': {'batch_size': 32, 'dataset_dir': './data/', 'graph_pkl_filename': './helper_files/adj_mat_volume.pkl', 'test_batch_size': 64, 'val_batch_size': 64}, 'log_level': 'INFO', 'model': {'cl_decay_steps': 2000, 'filter_type': 'dual_random_walk', 'horizon': 12, 'input_dim': 1, 'l1_decay': 0, 'max_diffusion_step': 1, 'num_nodes': 150, 'num_rnn_layers': 2, 'output_dim': 1, 'rnn_units': 64, 'seq_len': 12, 'use_curriculum_learning': True}, 'train': {'base_lr': 0.01, 'dropout': 0, 'epoch': 64, 'epochs': 300, 'epsilon': 0.001, 'global_step': 18655, 'groundtruth_file': 'y_truth_GCGRNN_volume_150.csv', 'log_dir': 'results', 'lr_decay_ratio': 0.1, 'max_grad_norm': 5, 'max_to_keep': 100, 'min_learning_rate': 2e-06, 'model_filename': './best_model\\models-462.8403-63427', 'optimizer': 'adam', 'patience': 50, 'preds_file': 'GCGRNN_volume_150.csv', 'steps': '[20]', 'test_every_n_epochs'

C:\Users\b\Desktop\direct_DCRNN\lib\utils.py:104: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(d, -1).flatten()


2021-03-19 15:30:08,723 - INFO - Total number of trainable parameters: 223168
INFO:tensorflow:Restoring parameters from ./best_model\models-462.8403-63427
2021-03-19 15:31:34,491 - INFO - Horizon 01, MAE: 244.94, MAPE: 0.0783, RMSE: 373.63
2021-03-19 15:31:34,518 - INFO - Horizon 02, MAE: 364.27, MAPE: 0.1250, RMSE: 562.18
2021-03-19 15:31:34,542 - INFO - Horizon 03, MAE: 440.97, MAPE: 0.1630, RMSE: 693.51
2021-03-19 15:31:34,563 - INFO - Horizon 04, MAE: 486.54, MAPE: 0.1881, RMSE: 773.55
2021-03-19 15:31:34,585 - INFO - Horizon 05, MAE: 506.08, MAPE: 0.1993, RMSE: 810.06
2021-03-19 15:31:34,606 - INFO - Horizon 06, MAE: 509.78, MAPE: 0.2011, RMSE: 820.19
2021-03-19 15:31:34,628 - INFO - Horizon 07, MAE: 512.07, MAPE: 0.2021, RMSE: 833.27
2021-03-19 15:31:34,649 - INFO - Horizon 08, MAE: 517.37, MAPE: 0.2027, RMSE: 858.46
2021-03-19 15:31:34,671 - INFO - Horizon 09, MAE: 519.25, MAPE: 0.2030, RMSE: 872.80
2021-03-19 15:31:34,692 - INFO - Horizon 10, MAE: 512.86, MAPE: 0.1964, RMSE: 87

In [8]:
# Look at the predictions, are they already scaled ?
outputs = np.load('./best_model/best_model_output.npz.')
for k in outputs:
    print(k)

predictions
groundtruth


In [9]:
print(np.max(outputs['predictions']))
print(np.min(outputs['predictions']))

11103.598
-160.9353


In [10]:
## seems like DCRNN predictions can be -ve